In [18]:
import os
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

In [19]:
# Define directories
image_directory = '../data/deigo/annotate-frames'  # Original Images
depth_map_root_directory = '../data/deigo/depth-maps/'  # Depth Map Images
json_file = '../data/deigo/diego-pipe-annotations/annotations/instances_default.json'  # JSON file

# Update the path to save results
results_csv_path = '../data/deigo/dmap-vid.csv'

In [20]:
# Mapping of images to their specific depth map directories
depth_map_directories = {
    'bream_oyster_frame_000001.png': os.path.join(depth_map_root_directory, 'bream-oyster'),
    'puffer_oyster_frame_000001.png': os.path.join(depth_map_root_directory, 'puffer-at-night-oyster'),
    'scat_oyster_frame_000001.png': os.path.join(depth_map_root_directory, 'scat-oyster-reef-day')
}

In [21]:

# Load the COCO annotations
coco = COCO(json_file)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [22]:
# Create a mapping from category_id to category_name
cats = coco.loadCats(coco.getCatIds())
category_mapping = {cat['id']: cat['name'] for cat in cats}
print("Category mapping:")
print(category_mapping)

Category mapping:
{1: '50', 2: '60', 3: '70', 4: '80', 5: '90'}


In [23]:
def load_and_normalize_depth_map(depth_map_path):
    # Load the depth map image
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_UNCHANGED)
    if depth_map is None:
        print(f"Failed to load depth map at {depth_map_path}.")
        return None
    
    # Normalize the depth map based on its data type
    if depth_map.dtype == np.uint8:
        # 8-bit depth map
        depth_map_normalized = depth_map / 255.0
    elif depth_map.dtype == np.uint16:
        # 16-bit depth map
        depth_map_normalized = depth_map / 65535.0
    else:
        # If depth map is already in float format between 0 and 1
        depth_map_normalized = depth_map

    return depth_map_normalized


In [24]:
def create_mask_from_annotation(ann, img_height, img_width):
    segmentation = ann['segmentation']
    
    if isinstance(segmentation, list):  # Polygon format
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        # Convert polygons to a mask
        for poly in segmentation:
            poly = np.array(poly).reshape((-1, 2))  # Reshape to (N, 2)
            cv2.fillPoly(mask, [np.int32(poly)], 1)  # Fill the polygon with 1's

    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        rle = segmentation
        if isinstance(rle['counts'], list):  # Uncompressed RLE
            rle = maskUtils.frPyObjects([rle], img_height, img_width)
        mask = maskUtils.decode(rle)  # Decode RLE to binary mask

    else:
        mask = None  # Segmentation format not recognized

    return mask


In [25]:
# List to store results
results = []

# Get all image IDs
image_ids = coco.getImgIds()
print(f"Total number of images: {len(image_ids)}")


Total number of images: 17


In [26]:
# Process each image
for img_id in image_ids:
    # Load image information
    img_info = coco.loadImgs(img_id)[0]
    img_name_with_ext = img_info['file_name']  # e.g., 'bream_oyster_frame_000001.png'
    img_name = os.path.splitext(img_name_with_ext)[0]  # e.g., 'bream_oyster_frame_000001'
    img_height = img_info['height']
    img_width = img_info['width']
    
    print(f"\nProcessing image ID {img_id}, name {img_name_with_ext}")

    # Determine depth map directory for the image
    if img_name_with_ext in depth_map_directories:
        # Use the specific depth map directory for this image
        depth_map_directory = depth_map_directories[img_name_with_ext]
        # Collect all depth maps in the directory
        depth_map_files = [f for f in os.listdir(depth_map_directory) if f.endswith('_pred04.png') and img_name in f]
        if not depth_map_files:
            print(f"No depth maps found for image {img_name_with_ext} in {depth_map_directory}")
            continue
    else:
        # Use the default depth map directory
        depth_map_directory = depth_map_root_directory
        depth_map_filename = f"{img_name}_pred04.png"
        depth_map_path = os.path.join(depth_map_directory, depth_map_filename)
        if not os.path.exists(depth_map_path):
            print(f"Depth map for image {img_name_with_ext} not found at {depth_map_path}")
            continue
        depth_map_files = [depth_map_filename]

    # Get annotation IDs for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    
    print(f"Number of annotations for image {img_name_with_ext}: {len(anns)}")
    
    # Process each depth map for the image
    for depth_map_file in depth_map_files:
        depth_map_path = os.path.join(depth_map_directory, depth_map_file)
        depth_map = load_and_normalize_depth_map(depth_map_path)
        if depth_map is None:
            continue  # Skip if depth map couldn't be loaded

        depth_map_folder_name = os.path.basename(depth_map_directory)
        depth_map_file_name = os.path.basename(depth_map_file)
        print(f"Processing depth map: {depth_map_file_name} from folder: {depth_map_folder_name}")

        # Process each annotation
        for ann in anns:
            annotation_id = ann['id']
            category_id = ann['category_id']
            category_name = category_mapping.get(category_id, "Unknown")
            print(f"Processing annotation ID {annotation_id}, category ID {category_id}, category name {category_name}")
            
            # Create mask for the annotation
            mask = create_mask_from_annotation(ann, img_height, img_width)
            if mask is None:
                print(f"Failed to create mask for annotation {annotation_id}")
                continue  # Skip if mask couldn't be created
            
            # Apply the mask to the depth map to extract depth values
            depth_values = depth_map[mask == 1]
            mean_depth = depth_values.mean() if depth_values.size > 0 else 0
            normalized_depth = mean_depth  # Depth map is already normalized between 0 and 1
            
            # Convert normalized depth to metric depth
            zmin = 0.4  # Minimum depth in meters (adjust if necessary)
            zmax = 20.0  # Maximum depth in meters (adjust if necessary)
            value_metric = (normalized_depth * (zmax - zmin)) + zmin
            
            print(f"Annotation ID {annotation_id}: mean depth {mean_depth}, normalized depth {normalized_depth}, value_metric {value_metric}")
            
            # Append the result to the list
            results.append({
                'image_name': img_name_with_ext,
                'image_id': img_id,
                'depth_map_folder': depth_map_folder_name,
                'depth_map_file': depth_map_file_name,
                'annotation_id': annotation_id,
                'category_id': category_id,
                'category_name': category_name,
                'normalized_depth': normalized_depth,
                'z-min': zmin,
                'z-max': zmax,
                'value_metric': value_metric
            })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print("\nResults DataFrame:")
print(results_df.head())

# Save the results to a CSV file
results_df.to_csv(results_csv_path, index=False)
print(f"\nResults saved to {results_csv_path}")


Processing image ID 1, name 1.png
Depth map for image 1.png not found at ../data/deigo/depth-maps/1_pred04.png

Processing image ID 2, name 10.png
Depth map for image 10.png not found at ../data/deigo/depth-maps/10_pred04.png

Processing image ID 3, name 2.png
Depth map for image 2.png not found at ../data/deigo/depth-maps/2_pred04.png

Processing image ID 4, name 3.png
Depth map for image 3.png not found at ../data/deigo/depth-maps/3_pred04.png

Processing image ID 5, name 4.png
Depth map for image 4.png not found at ../data/deigo/depth-maps/4_pred04.png

Processing image ID 6, name 5.png
Depth map for image 5.png not found at ../data/deigo/depth-maps/5_pred04.png

Processing image ID 7, name 6.png
Depth map for image 6.png not found at ../data/deigo/depth-maps/6_pred04.png

Processing image ID 8, name 7.png
Depth map for image 7.png not found at ../data/deigo/depth-maps/7_pred04.png

Processing image ID 9, name 8.png
Depth map for image 8.png not found at ../data/deigo/depth-maps/8_